# Instructions

* This notebook is refered to MScFE 690 Capstone - Module 4 - Design and it is finished and running succefully, for an ongoing updated code go to the vLive version.
* Since the notebook requires the instalation of modules it is required to restart the sessions afer the modules are installed.


In [8]:
import requests
import pandas as pd
import os
import time
from datetime import date

import json
import re
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.dates as mdates
from plotly.subplots import make_subplots

from plotly.subplots import make_subplots
import os
import requests
import numpy as np

try:
  import quiverquant
except:
  !pip install quiverquant
  import quiverquant



In [9]:
# Create output directory
os.makedirs("data", exist_ok=True)

# Data collection

## Configuration

In [26]:
POLYGON_KEY = "ft2rquZoprD0MSBmJqQfD_8qNsxrkK_1"

In [ ]:
# Get the directory of this script
SCRIPT_DIR = os.getcwd()
# Congressional term date ranges
CONGRESS_START_DATE = "2019-01-03"  # 116th Congress start
CONGRESS_END_DATE = "2025-01-02"    # 118th Congress end

# Major economic states for geographic filtering
MAJOR_STATES = ['CA', 'NY', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI',
                'NJ', 'VA', 'WA', 'AZ', 'MA', 'TN', 'CO', 'MD', 'CT', 'AL']

# Output directory
OUTPUT_DIR = os.path.join(SCRIPT_DIR, 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [14]:
lawmakers = [
    # --- Senators ---
    {"name": "Mitch McConnell", "party": "Republican", "chamber": "Senate", "state": "KY", "bioguide": "M000355"},
#    {"name": "Chuck Grassley", "party": "Republican", "chamber": "Senate", "state": "IA", "bioguide": "G000386"}, No reported trade
    {"name": "Ron Wyden", "party": "Democrat", "chamber": "Senate", "state": "OR", "bioguide": "W000779"},
    {"name": "Patty Murray", "party": "Democrat", "chamber": "Senate", "state": "WA", "bioguide": "M001111"},
    {"name": "Dick Durbin", "party": "Democrat", "chamber": "Senate", "state": "IL", "bioguide": "D000563"},
    {"name": "Ted Cruz", "party": "Republican", "chamber": "Senate", "state": "TX", "bioguide": "C001098"},  # state fixed from "ME"
    {"name": "Maria Cantwell", "party": "Democrat", "chamber": "Senate", "state": "WA", "bioguide": "C000127"},
    {"name": "Mitt Romney", "party": "Republican", "chamber": "Senate", "state": "UT", "bioguide": "R000615"},
    {"name": "John Cornyn", "party": "Republican", "chamber": "Senate", "state": "TX", "bioguide": "C001056"},
    {"name": "Bernie Sanders", "party": "Independent", "chamber": "Senate", "state": "VT", "bioguide": "S000033"},

    # --- Representatives ---
    {"name": "Nancy Pelosi", "party": "Democrat", "chamber": "House", "state": "CA", "bioguide": "P000197"},
    {"name": "Kevin McCarthy", "party": "Republican", "chamber": "House", "state": "CA", "bioguide": "M001165"},
    {"name": "Patrick McHenry", "party": "Republican", "chamber": "House", "state": "NC", "bioguide": "M001156"},
    {"name": "Maxine Waters", "party": "Democrat", "chamber": "House", "state": "CA", "bioguide": "W000187"},
    {"name": "Richard Neal", "party": "Democrat", "chamber": "House", "state": "MA", "bioguide": "N000015"},
    {"name": "Frank Pallone Jr.", "party": "Democrat", "chamber": "House", "state": "NJ", "bioguide": "P000034"},
    {"name": "Gregory Meeks", "party": "Democrat", "chamber": "House", "state": "NY", "bioguide": "M001137"},
    {"name": "Adam Schiff", "party": "Democrat", "chamber": "House", "state": "CA", "bioguide": "S001150"},
    {"name": "Steny Hoyer", "party": "Democrat", "chamber": "House", "state": "MD", "bioguide": "H000874"},
    {"name": "Jim Jordan", "party": "Republican", "chamber": "House", "state": "OH", "bioguide": "J000289"},
]
lawmakers = pd.DataFrame(lawmakers)
lawmakers.to_csv("data/lawmakers.csv", index=False)



## Yfinance data

In [10]:
# 1. Fetch Data
ticker = "NVDA"
df = yf.download(ticker, period="1y", interval="1d")
df = df.stack(level='Ticker').reset_index()

# 2. Create Subplots (2 Rows: Price on top, Volume on bottom)
# row_heights=[0.7, 0.3] gives 70% space to Price, 30% to Volume
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=(f'{ticker} Price', 'Volume'),
    row_heights=[0.7, 0.3]
)

# 3. Add Candlestick Trace (Row 1)
fig.add_trace(go.Candlestick(
    x=df.Date,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Price'
), row=1, col=1)

# 4. Add Volume Trace (Row 2)
# We color the volume bars based on price change (Green for up, Red for down)
colors = ['green' if row['Open'] - row['Close'] >= 0
          else 'red' for index, row in df.iterrows()]

fig.add_trace(go.Bar(
    x=df.Date,
    y=df['Volume'],
    marker_color=colors,
    name='Volume'
), row=2, col=1)

# 5. Add Moving Average (Optional, on Row 1)
df['MA20'] = df['Close'].rolling(window=20).mean()
fig.add_trace(go.Scatter(
    x=df.Date,
    y=df['MA20'],
    line=dict(color='orange', width=2),
    name='20-Day MA'
), row=1, col=1)

# 6. Final Layout Adjustments
fig.update_layout(
    title=f'{ticker} Candlestick Chart with Volume',
    template='plotly_dark',
    xaxis_rangeslider_visible=False, # Hides the slider (redundant with zoom)
    height=800 # Taller figure to accommodate both plots
)

# Show the Plot
fig.show()

/tmp/ipython-input-1349361877.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1349361877.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level='Ticker').reset_index()


In [11]:
df = yf.download(ticker, period="1y", interval="1d", ignore_tz=True)

/tmp/ipython-input-462295115.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [12]:
clean_df = df.stack(level='Ticker').reset_index()
clean_df.head()

/tmp/ipython-input-1590771923.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Price,Date,Ticker,Close,High,Low,Open,Volume
0,2025-02-06,NVDA,128.644806,128.734793,125.175761,127.385156,251483600
1,2025-02-07,NVDA,129.804489,130.334343,124.965816,129.184664,228186300
2,2025-02-10,NVDA,133.533478,134.963079,129.924464,130.054419,216989100
3,2025-02-11,NVDA,132.763702,134.443236,130.984190,132.543761,178902400
4,2025-02-12,NVDA,131.104141,132.203847,129.044707,129.984452,160278600


## Quiver transaction history

In [13]:
class CustomQuiver(quiverquant.quiver):

    def congress_trading(self, ticker="", politician=True, recent=False, representative="", bioguide=""):


      if recent:
          urlStart = 'https://api.quiverquant.com/beta/live/congresstrading'
      else:
          urlStart = 'https://api.quiverquant.com/beta/bulk/congresstrading'

      if politician:
          representative = representative.replace(" ", "%20")
          ticker = ticker.replace(" ", "%20")

          if representative:
            url = urlStart+"?representative="+representative+ticker

          elif bioguide:
            url = urlStart+"?bioguide_id="+bioguide+ticker


      elif len(ticker)>0:
          url = urlStart+"/"+ticker
      else:
          url = urlStart
      print(url)
      r = requests.get(url, headers=self.headers)
      j = json.loads(r.content)
      df = pd.DataFrame(j)
      return df

# === API KEYS  ===
# - Quiver Quantitative: https://www.quiverquant.com/ (paid subscription required for API access, but has free trial)
# - Documentation on https://api.quiverquant.com/docs/#/ and https://github.com/Quiver-Quantitative/python-api

QUIVER_TOKEN = "a8dcf66a9fd588b78513ccedb4b92acc948ac6e3"

# # 1 Initialize Quiver client (token required for full access)

# qq = quiverquant.quiver(QUIVER_TOKEN)

quiver = CustomQuiver(QUIVER_TOKEN)


In [ ]:
def fetch_congress_trading(use_cache=True, cache_file=None):
    """
    Fetch all congress trading data from Quiver API.

    Parameters:
    -----------
    use_cache : bool
        If True, use cached data if available
    cache_file : str
        Path to cache file

    Returns:
    --------
    pd.DataFrame : All congress trading records
    """
    if cache_file is None:
        cache_file = os.path.join(OUTPUT_DIR, 'raw_congress_trades.csv')

    # Check cache
    if use_cache and os.path.exists(cache_file):
        print(f"Loading cached data from {cache_file}")
        df = pd.read_csv(cache_file)
        return df

    # Fetch from API
    print("Fetching congress trading data from Quiver API...")
    quiver = CustomQuiver(QUIVER_TOKEN)
    df = quiver.congress_trading(recent=False)

    # Save cache
    df.to_csv(cache_file, index=False)
    print(f"Saved {len(df)} records to {cache_file}")

    return df

In [ ]:
raw_df = fetch_congress_trading(use_cache=True)

In [ ]:
def filter_by_date_range(df, start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE):
    """
    Filter trading data to specified date range.

    Parameters:
    -----------
    df : pd.DataFrame
        Trading data
    start_date : str
        Start date (YYYY-MM-DD)
    end_date : str
        End date (YYYY-MM-DD)

    Returns:
    --------
    pd.DataFrame : Filtered data
    """
    df_filtered = df.loc[
        (df['Traded'] >= start_date) &
        (df['Traded'] <= end_date)
    ].copy()

    print(f"Filtered to {len(df_filtered)} records between {start_date} and {end_date}")
    return df_filtered


In [ ]:
def clean_trading_data(df):
    """
    Clean and preprocess trading data.

    Parameters:
    -----------
    df : pd.DataFrame
        Raw trading data

    Returns:
    --------
    pd.DataFrame : Cleaned data
    """
    df = df.copy()

    # Convert numeric columns
    df['Trade_Size_USD'] = pd.to_numeric(df['Trade_Size_USD'], errors='coerce')
    df['excess_return'] = pd.to_numeric(df['excess_return'], errors='coerce')

    # Drop rows with missing critical values
    df = df.dropna(subset=['Trade_Size_USD', 'excess_return'])

    # Filter out non-positive trade sizes
    df = df[df['Trade_Size_USD'] > 0]

    # Convert dates
    df['Traded'] = pd.to_datetime(df['Traded'])
    df['Filed'] = pd.to_datetime(df['Filed'], errors='coerce')

    # Extract state from District (e.g., 'CA11' -> 'CA')
    df['State'] = df['District'].astype(str).str.strip().str.extract(r'([A-Z]{2})', expand=False)

    # Add year column for yearly analysis
    df['Year'] = df['Traded'].dt.year

    print(f"Cleaned data: {len(df)} records, {df['Name'].nunique()} unique politicians")
    return df



In [ ]:
df_date_filtered = filter_by_date_range(raw_df)

In [ ]:
cleaned_df = clean_trading_data(df_date_filtered)

In [ ]:
df_filtered = filter_by_major_states(cleaned_df)

# EDA

In [15]:
class Congressman:
  def __init__(self, name:str, party:str, chamber:str, state:str, bioguide:str, quiverClient):

    self.name = name
    self.party = party
    self.chamber = chamber
    self.state = state
    self.bioguide = bioguide
# -------------

    self.transactions = self.get_transactions(quiverClient)
    self.tickers = self.get_tickers()

    self.positions = self.get_positions()
    self.positions45days = self.get_positions(delay=True)

    self.portfolio = None
    self.portfolio45days = None

    self.weights = None
    self.weights45days = None

    self.returns = None
    self.returns45days = None

  def get_transactions(self, quiverClient):

    transactions = quiverClient.congress_trading(bioguide=self.bioguide)
    if transactions.empty:
        print("No transacction history for ", self.name)
        return None

    transactions = transactions[~transactions['TickerType'].str.contains('OP|None', na=True)]
    transactions = transactions[['Ticker', 'TickerType', 'Traded', 'Transaction', 'Description', 'Filed', 'Trade_Size_USD']]

    # Removing options trades
    transactions = transactions[~transactions['Description'].str.contains('options', na=False, case=False)]

    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'].str.extract(r'(\d{1,3}(?:,\d{3})*)', expand=False)

    # Remove commas from the extracted strings and convert to a numeric type (float)
    # Errors='coerce' will turn values that can't be converted into NaN
    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'].str.replace(',', '', regex=False).astype(float)

    transactions['Transaction'] = transactions['Transaction'].map({"Purchase": 1, "Sale": -1})
    transactions['Transaction'].dropna

    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'] * transactions['Transaction']

    transactions["Traded"] = pd.to_datetime(transactions["Traded"])
    transactions = transactions.set_index('Traded')

    return transactions

  def get_tickers(self):

    tickers = self.transactions['Ticker'].unique().tolist()

    # Mapping for known ticker changes (keep this)
    ticker_mapping = {'FB': 'META'}  # add more if needed
    tickers = [ticker_mapping.get(t, t) for t in tickers]

    # Strict filter: only allow typical US stock ticker patterns
    # - Uppercase letters, numbers, and dots (for class shares like BRK.B)
    # - No spaces, no commas, no lowercase, reasonable length
    def is_valid_ticker(t):
        return (
            bool(re.fullmatch(r'[A-Z0-9\.]{1,10}', t)) and
            ' ' not in t and
            ',' not in t and
            t.isupper()
        )

    valid_tickers = [t for t in tickers if is_valid_ticker(t)]

    manual_excludes = {'BFET', 'WORK', 'VSNT'}
    valid_tickers = [t for t in valid_tickers if t not in manual_excludes]

    print(valid_tickers)
    return valid_tickers

  def get_positions(self, delay=False):

    if self.transactions.empty:
      # Fallback empty DataFrame
      print('No positions for: ', self.name)
      return pd.DataFrame(columns=self.tickers)

    trans = self.transactions.copy()

    # Apply 45-day delay if requested
    trade_date_col = "Traded"  # default uses original index

    if delay:
        print(f"Delayed for: {self.name}")
        trans['TradedDelay45'] = trans.index + pd.Timedelta(days=45)
        trade_date_col = "TradedDelay45"

    # Daily net dollar flows by ticker
    daily_delta = trans.pivot_table(
        index=trade_date_col,
        columns="Ticker",
        values="Trade_Size_USD",
        aggfunc="sum"
    ).sort_index()

    print(f"Daily delta for {self.name}:")
    positions = daily_delta.cumsum()

    full_index = pd.date_range(
        start=positions.index.min(),
        end=positions.index.max(),
        freq="D"
    )

    # Reindex to daily, forward-fill holdings, zero before first trade
    positions = (
        positions
        .reindex(full_index)
        .ffill()
        .fillna(0)
    )

    start_date_str = '2019-01-03'
    end_date_str = '2025-01-03'
    start = date.fromisoformat(start_date_str)
    end = date.fromisoformat(end_date_str)

    # Final safety: only our tickers, sorted columns
    target_index = pd.date_range(start=start, end=end, freq='D')
    positions = positions.reindex(target_index)

    positions = positions.loc[:, positions.columns.isin(self.tickers)]

    return positions

# -----

lawmakers_test = [
    # --- Senators ---
    {"name": "Mitch McConnell", "party": "Republican", "chamber": "Senate", "state": "KY", "bioguide": "M000355"},
    {"name": "Nancy Pelosi", "party": "Democrat", "chamber": "House", "state": "CA", "bioguide": "P000197"},
    {"name": "Ted Cruz", "party": "Republican", "chamber": "Senate", "state": "TX", "bioguide": "C001098"},  # state fixed from "ME"
  ]

congressmen = []

for l in lawmakers_test:
    congressman = Congressman(
        name=l["name"],
        party=l["party"],
        chamber=l["chamber"],
        state=l["state"],
        bioguide=l["bioguide"],
        quiverClient=quiver
    )
    congressmen.append(congressman)



all_tickers = [ticker for c in congressmen for ticker in c.tickers]
unique_tickers = list(set(all_tickers))  # removes duplicates
unique_tickers.sort()                    # optional: alphabetical order


def get_prices(tickers):
  start_date_str = '2019-01-03'
  end_date_str = '2025-01-03'
  start_date = date.fromisoformat(start_date_str)
  end_date = date.fromisoformat(end_date_str)

  all_ticker_prices = {}

  if tickers:
      print(f"Downloading prices for {len(tickers)} tickers...")
      prices_multi = yf.download(
          tickers,
          start=start_date_str,
          end=end_date_str,
          interval='1d',
          auto_adjust=True,
          progress=True
      )

      # prices_multi['Close'] is a DataFrame with columns = tickers
      close_df = prices_multi['Close']

      for ticker in tickers:
          series = close_df[ticker]
          if series.notna().any():  # has at least some real data
              all_ticker_prices[ticker] = series
          else:
              print(f"No valid price data for {ticker}")

  else:
      print("No valid tickers found.")

  # Create a master DataFrame with a full date range
  full_date_range = pd.date_range(start=start_date_str, end=end_date_str, freq='D')
  prices_df = pd.DataFrame(index=full_date_range)

  # Populate the master DataFrame with ticker prices
  for ticker, prices_series in all_ticker_prices.items():
      prices_df[ticker] = prices_series

  prices_df = prices_df.dropna(how='all')
  prices_df = prices_df.fillna(0)

  return prices_df

prices = get_prices(unique_tickers)
print(prices)


https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=M000355
['LAZR', 'WFC', 'KR', 'VMC', 'IR']
Daily delta for Mitch McConnell:
Delayed for: Mitch McConnell
Daily delta for Mitch McConnell:
https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=P000197
['AB', 'GOOGL', 'AAPL', 'DIS', 'PYPL', 'NVDA', 'AMZN', 'MSFT', 'V', 'TSLA', 'NFLX', 'RBLX', 'WBD', 'CRWD', 'META', 'MORN', 'IBKR', 'T']
Daily delta for Nancy Pelosi:
Delayed for: Nancy Pelosi
Daily delta for Nancy Pelosi:
https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=C001098


[                       0%                       ]

['GS', 'BTC', 'PAGP', 'OKE']
Daily delta for Ted Cruz:
Delayed for: Ted Cruz
Daily delta for Ted Cruz:


[*********************100%***********************]  27 of 27 completed


                  AAPL         AB        AMZN        BTC        CRWD  \
2019-01-03   33.799664  15.191700   75.014000   0.000000    0.000000   
2019-01-04   35.242565  15.540110   78.769501   0.000000    0.000000   
2019-01-07   35.164116  15.462683   81.475502   0.000000    0.000000   
2019-01-08   35.834461  15.540110   82.829002   0.000000    0.000000   
2019-01-09   36.442986  15.517989   82.971001   0.000000    0.000000   
...                ...        ...         ...        ...         ...   
2024-12-26  257.853760  34.239243  227.050003  42.330002  365.079987   
2024-12-27  254.439209  34.083652  223.750000  41.820000  354.989990   
2024-12-30  251.064499  33.369759  221.300003  41.750000  349.309998   
2024-12-31  249.292511  33.946365  219.389999  41.869999  342.160004   
2025-01-02  242.752090  33.918903  220.220001  43.160000  347.339996   

                   DIS       GOOGL          GS       IBKR         IR  ...  \
2019-01-03  102.629341   50.888996  144.015793  12.552105 

In [16]:
congressmen[0].positions

Ticker,IR,KR,LAZR,VMC,WFC
2019-01-03,0.0,0.0,0.0,0.0,600.0
2019-01-04,0.0,0.0,0.0,0.0,600.0
2019-01-05,0.0,0.0,0.0,0.0,600.0
2019-01-06,0.0,0.0,0.0,0.0,600.0
2019-01-07,0.0,0.0,0.0,0.0,600.0
...,...,...,...,...,...
2024-12-30,0.0,100.0,0.0,0.0,2900.0
2024-12-31,0.0,100.0,0.0,0.0,2900.0
2025-01-01,0.0,100.0,0.0,0.0,2900.0
2025-01-02,0.0,100.0,0.0,0.0,2900.0


In [20]:
congressmen[1].transactions

,Ticker,TickerType,Transaction,Description,Filed,Trade_Size_USD
Traded,,,,,,
2026-01-16,AB,AB,1.0,"PURCHASED 25,000 SHARES.",2026-01-23,100.0
2026-01-02,VSNT,ST,NaN,776 SHARES AND CASH IN LIEU RECEIVED AS A RESU...,2026-01-23,NaN
2025-12-30,GOOGL,ST,-1.0,"CONTRIBUTION OF 7,704 SHARES HELD PERSONALLY T...",2026-01-23,-100.0
2025-12-30,AAPL,ST,-1.0,"CONTRIBUTION OF 28,200 SHARES TO DONOR-ADVISED...",2026-01-23,-500.0
2025-12-30,DIS,ST,-1.0,"SOLD 10,000 SHARES.",2026-01-23,-100.0
2025-12-30,PYPL,ST,-1.0,"SOLD 5,000 SHARES.",2026-01-23,-250.0
2025-12-24,AAPL,ST,-1.0,"SOLD 45,000 SHARES.",2026-01-23,-500.0
2025-12-24,NVDA,ST,-1.0,"SOLD 20,000 SHARES.",2026-01-23,-100.0
2025-12-24,AMZN,ST,-1.0,None,2026-01-23,-100.0


In [23]:
congressmen[2].transactions

,Ticker,TickerType,Transaction,Description,Filed,Trade_Size_USD
Traded,,,,,,
2025-11-11,GS,Stock,NaN,None,2025-11-12,NaN
2024-04-15,GS,Stock,NaN,None,2024-04-24,NaN
2022-01-25,BTC,Cryptocurrency,1.0,Exchange/Platform : River,2022-02-04,150.0
2019-11-27,PAGP,Stock,NaN,None,2019-12-03,NaN
2019-05-15,GS,Stock,NaN,None,2019-05-24,NaN
2016-04-17,OKE,Stock,NaN,None,2016-05-17,NaN
2016-02-25,OKE,Stock,NaN,None,2016-03-28,NaN
2016-02-25,GS,Stock,NaN,None,2016-03-28,NaN
2015-02-03,PAGP,Stock,1.0,None,2015-02-09,500.0


In [25]:
congressmen[2].name

'Ted Cruz'

In [28]:
nancy = quiver.congress_trading(bioguide="P000197")

https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=P000197


In [29]:
nancy

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
0,VST,ST,None,2026-01-16,Purchase,100001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-13.4530530601148,None,2026-01-26
1,GOOGL,ST,None,2026-01-16,Purchase,500001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-2.14571194184928,None,2026-01-26
2,AB,AB,None,2026-01-16,Purchase,1000001.0,None,None,"PURCHASED 25,000 SHARES.",Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,9.38078906523425,None,2026-01-26
3,AMZN,ST,None,2026-01-16,Purchase,500001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-5.69032501807829,None,2026-01-26
4,NVDA,ST,None,2026-01-16,Purchase,250001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-4.69045947626012,None,2026-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,SFLY,None,None,2014-10-23,Sale,15001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-33.0655511900991,None,2023-11-16
178,BCOR,None,None,2014-10-23,Sale,1001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-213.268050292592,None,2023-11-16
179,BRCM,None,None,2014-10-23,Sale,1001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,39.4159548641353,None,2023-11-16
180,RHI,None,None,2014-09-22,Sale,100001.0,None,None,CONTRIBUTION OF SHARES HELD PERSONALLY TO THE ...,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-279.620720124192,None,2023-11-16


In [30]:
nancy.Trade_Size_USD = pd.to_numeric(nancy.Trade_Size_USD, errors='coerce')

In [31]:
nancy

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
0,VST,ST,None,2026-01-16,Purchase,100001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-13.4530530601148,None,2026-01-26
1,GOOGL,ST,None,2026-01-16,Purchase,500001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-2.14571194184928,None,2026-01-26
2,AB,AB,None,2026-01-16,Purchase,1000001.0,None,None,"PURCHASED 25,000 SHARES.",Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,9.38078906523425,None,2026-01-26
3,AMZN,ST,None,2026-01-16,Purchase,500001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-5.69032501807829,None,2026-01-26
4,NVDA,ST,None,2026-01-16,Purchase,250001.0,None,None,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,None,None,-4.69045947626012,None,2026-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,SFLY,None,None,2014-10-23,Sale,15001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-33.0655511900991,None,2023-11-16
178,BCOR,None,None,2014-10-23,Sale,1001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-213.268050292592,None,2023-11-16
179,BRCM,None,None,2014-10-23,Sale,1001.0,None,None,None,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,39.4159548641353,None,2023-11-16
180,RHI,None,None,2014-09-22,Sale,100001.0,None,None,CONTRIBUTION OF SHARES HELD PERSONALLY TO THE ...,Ms. Nancy Pelosi,P000197,2014-11-05,D,CA12,Representatives,None,None,-279.620720124192,None,2023-11-16
